In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,ow2xPaj,0,MASC,40.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,52KOZq,0,MASC,40.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,JBrbYvE,0,MASC,27.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,NDrRZl,0,MASC,29.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,MVr8z8x,0,MASC,38.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12344730


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
3407986,1112304125,YlMLGD,1,FEM,43.0,-,-,Graduado,Graduado,Graduado,...,False,True,False,False,False,False,False,False,True,NaN
11761707,1112302280,EXwMz9,0,MASC,34.0,-,-,-,-,-,...,False,False,True,True,False,True,False,False,False,NaN
9405195,1112306953,aaLO9W,0,FEM,34.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
8514808,1112368385,5688zG,0,FEM,47.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
11664360,1112463328,YZjeRZ,0,MASC,46.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,NaN
6017853,1112430005,VNJM1rm,1,FEM,27.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,1.0
3629973,1112277176,dYjAZNb,1,MASC,25.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,NaN
2071638,1112206678,bOzVzQ0,1,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
8432005,1112296005,5Qbxjv,0,MASC,34.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
8959173,1112268224,JBxabBJ,0,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7564689,1112298993,96MNjMZ,0,FEM,23.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,True,0.0
5834712,1112256007,YvxNBD,1,FEM,38.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.0
11679437,1112369522,mzd65k6,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
2618214,1112296264,JBXmxWr,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
414402,1112378885,8VRKbW,1,FEM,30.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.0
11503253,1112384982,E2BLo4,0,FEM,35.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
2298311,1112320527,QNrVb9N,1,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
63824,1112373509,BmBmokL,1,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
839803,1111970596,EzpGqV8,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
735799,1112433181,xkarzDY,1,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,2.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  11727493 Test:  617237


In [12]:
import gc
gc.collect()

379

In [13]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=4096,max_depth=13,n_estimators=1000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

74.09001728671483


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV7.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV7.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
98671,98554,1112506993,aBYQ89,FEM,29.0,-,Graduado,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
34391,7512,1111716604,ZWR2Nz,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
5979,74298,1112463686,LNK2WAP,MASC,39.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,1.0
56413,66381,1112459509,8Pb4vR,FEM,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
53021,57520,1112451476,1kK823,FEM,32.0,-,-,En Curso,-,-,...,False,False,False,False,False,False,False,False,False,0.0
96754,96415,1112494019,6Vp9vO,MASC,40.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
91352,81495,1112466058,Y0YP4w,FEM,42.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
26415,46169,1112429754,LNwEzlw,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,2.0
94964,94334,1112483816,e95YLp,FEM,36.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
81312,28976,1112367877,EOwj18,MASC,64.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.60293479, 0.39706521],
       [0.56234362, 0.43765638],
       [0.56449734, 0.43550266],
       ...,
       [0.82842402, 0.17157598],
       [0.59049188, 0.40950812],
       [0.93775201, 0.06224799]])

In [18]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [19]:
prediccion['sepostulo']=y_final2

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.437181
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.404494
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.396515
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.223281
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.317451


In [20]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.397065
1,1,0.437656
2,2,0.435503
3,3,0.206046
4,4,0.415978


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion20.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.5843051760731877